# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [52]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. '''

## 2. Generate the mathematical model

In [53]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [54]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Production Planning

**Parameters:**

* **Availability of components:**
    *  A1 = 10,000 l (Available quantity of C1)
    *  A2 = 15,000 l (Available quantity of C2)
* **Minimum demand:**
    * D_GCA = 6,000 l (Minimum demand for GCA)
    * D_GCB = 7,000 l (Minimum demand for GCB)
    * D_GCC = 9,000 l (Minimum demand for GCC)
* **Critical element proportion:**
    * E1 = 0.4 (Proportion of critical element in C1)
    * E2 = 0.2 (Proportion of critical element in C2)
* **Critical element requirements:**
    * R_GCA = 0.3 (Minimum proportion of critical element in GCA)
    * R_GCB = 0.3 (Maximum proportion of critical element in GCB)
* **Component ratio:**
    * CR_GCC = 0.3 (Minimum ratio of C1 to C2 in GCC)
* **Profit per liter:**
    * P_GCA = $120 (Profit for GCA)
    * P_GCB = $135 (Profit for GCB)
    * P_GCC = $155 (Profit for GCC)

**Decision Variables:**

* **X_ij:** Liters of component i (i = 1 for C1, i = 2 for C2) used in product j (j = GCA, GCB, GCC).

**Objective Function:**

Maximize total profit:

```
Maximize Z = P_GCA * (X_1GCA + X_2GCA) + P_GCB * (X_1GCB + X_2GCB) + P_GCC * (X_1GCC + X_2GCC)
```

**Constraints:**

1. **Availability constraints:**
    * X_1GCA + X_1GCB + X_1GCC <= A1 
    * X_2GCA + X_2GCB + X_2GCC <= A2
2. **Demand constraints:**
    * X_1GCA + X_2GCA >= D_GCA
    * X_1GCB + X_2GCB >= D_GCB
    * X_1GCC + X_2GCC >= D_GCC
3. **Critical element constraints:**
    * (E1 * X_1GCA + E2 * X_2GCA) / (X_1GCA + X_2GCA) >= R_GCA 
    * (E1 * X_1GCB + E2 * X_2GCB) / (X_1GCB + X_2GCB) <= R_GCB
4. **Component ratio constraint:**
    * X_1GCC / X_2GCC >= CR_GCC
5. **Non-negativity constraints:**
    * X_ij >= 0 for all i and j

This model can be solved using linear programming software to determine the optimal production plan for the firm, maximizing its profit while meeting all the specified requirements. 


## 3. Generate the pyomo code

In [55]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [56]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets for components and products
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]

# Define parameters (using the provided sample data)
availability = {"C1": 10000, "C2": 15000}
demand = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
critical_proportion = {"C1": 0.4, "C2": 0.2}
critical_requirement = {"GCA": 0.3, "GCB": 0.3}
component_ratio = {"GCC": 0.3}
profit = {"GCA": 120, "GCB": 135, "GCC": 155}

# Define decision variables
model.X = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Define the objective function
model.profit = pyo.Objective(
    expr=sum(
        profit[j] * (model.X[i, j] for i in components) for j in products
    ),
    sense=pyo.maximize,
)

# Define constraints
# 1. Availability constraints
model.availability_constraints = pyo.ConstraintList()
for i in components:
    model.availability_constraints.add(
        sum(model.X[i, j] for j in products) <= availability[i]
    )

# 2. Demand constraints
model.demand_constraints = pyo.ConstraintList()
for j in products:
    model.demand_constraints.add(
        sum(model.X[i, j] for i in components) >= demand[j]
    )

# 3. Critical element constraints
model.critical_element_constraints = pyo.ConstraintList()
model.critical_element_constraints.add(
    (
        sum(critical_proportion[i] * model.X[i, "GCA"] for i in components)
        / sum(model.X[i, "GCA"] for i in components)
    )
    >= critical_requirement["GCA"]
)
model.critical_element_constraints.add(
    (
        sum(critical_proportion[i] * model.X[i, "GCB"] for i in components)
        / sum(model.X[i, "GCB"] for i in components)
    )
    <= critical_requirement["GCB"]
)

# 4. Component ratio constraint
model.component_ratio_constraint = pyo.Constraint(
    expr=model.X["C1", "GCC"] / model.X["C2", "GCC"] >= component_ratio["GCC"]
)

# Solve the model
solver = pyo.SolverFactory("glpk")
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"X_{i}{j}: {model.X[i, j].value:.2f}")

print(f"Total Profit: ${model.profit():.2f}")
``` 


## 4. Input problem data and try running the generated code

In [57]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets for components and products
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]

# Define parameters (using the provided sample data)
availability = {"C1": 10000, "C2": 15000}
demand = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
critical_proportion = {"C1": 0.4, "C2": 0.2}
critical_requirement = {"GCA": 0.3, "GCB": 0.3}
component_ratio = {"GCC": 0.3}
profit = {"GCA": 120, "GCB": 135, "GCC": 155}

# Define decision variables
model.X = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Define the objective function
model.profit = pyo.Objective(
    expr=sum(
        profit[j] * (model.X[i, j] for i in components) for j in products
    ),
    sense=pyo.maximize,
)

# Define constraints
# 1. Availability constraints
model.availability_constraints = pyo.ConstraintList()
for i in components:
    model.availability_constraints.add(
        sum(model.X[i, j] for j in products) <= availability[i]
    )

# 2. Demand constraints
model.demand_constraints = pyo.ConstraintList()
for j in products:
    model.demand_constraints.add(
        sum(model.X[i, j] for i in components) >= demand[j]
    )

# 3. Critical element constraints
model.critical_element_constraints = pyo.ConstraintList()
model.critical_element_constraints.add(
    (
        sum(critical_proportion[i] * model.X[i, "GCA"] for i in components)
        / sum(model.X[i, "GCA"] for i in components)
    )
    >= critical_requirement["GCA"]
)
model.critical_element_constraints.add(
    (
        sum(critical_proportion[i] * model.X[i, "GCB"] for i in components)
        / sum(model.X[i, "GCB"] for i in components)
    )
    <= critical_requirement["GCB"]
)

# 4. Component ratio constraint
model.component_ratio_constraint = pyo.Constraint(
    expr=model.X["C1", "GCC"] / model.X["C2", "GCC"] >= component_ratio["GCC"]
)

# Solve the model
solver = pyo.SolverFactory("glpk")
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"X_{i}{j}: {model.X[i, j].value:.2f}")

print(f"Total Profit: ${model.profit():.2f}")

TypeError: unsupported operand type(s) for *: 'int' and 'generator'

## 5. Correct the code to verify model viability (optional)

In [59]:
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets for components and products
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]

# Define parameters (using the provided sample data)
availability = {"C1": 10000, "C2": 15000}
demand = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
critical_proportion = {"C1": 0.4, "C2": 0.2}
critical_requirement = {"GCA": 0.3, "GCB": 0.3}
component_ratio = {"GCC": 0.3}
profit = {"GCA": 120, "GCB": 135, "GCC": 155}

# Define decision variables
model.X = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Define the objective function
model.profit = pyo.Objective(
    expr=sum(
        profit[j] * sum(model.X[i, j] for i in components) for j in products
    ),
    sense=pyo.maximize,
)

# Define constraints
# 1. Availability constraints
model.availability_constraints = pyo.ConstraintList()
for i in components:
    model.availability_constraints.add(
        sum(model.X[i, j] for j in products) <= availability[i]
    )

# 2. Demand constraints
model.demand_constraints = pyo.ConstraintList()
for j in products:
    model.demand_constraints.add(
        sum(model.X[i, j] for i in components) >= demand[j]
    )

# 3. Critical element constraints
model.critical_element_constraints = pyo.ConstraintList()
model.critical_element_constraints.add(
    (
        sum(critical_proportion[i] * model.X[i, "GCA"] for i in components)
        / sum(model.X[i, "GCA"] for i in components)
    )
    >= critical_requirement["GCA"]
)
model.critical_element_constraints.add(
    (
        sum(critical_proportion[i] * model.X[i, "GCB"] for i in components)
        / sum(model.X[i, "GCB"] for i in components)
    )
    <= critical_requirement["GCB"]
)

# 4. Component ratio constraint
model.component_ratio_constraint = pyo.Constraint(
    expr=model.X["C1", "GCC"] / model.X["C2", "GCC"] >= component_ratio["GCC"]
)

# Solve the model
solver = pyo.SolverFactory("ipopt")
solver.solve(model)

# Print the results
print("Optimal Solution:")
for i in components:
    for j in products:
        print(f"X_{i}{j}: {model.X[i, j].value:.2f}")

print(f"Total Profit: ${model.profit():.2f}")

Optimal Solution:
X_C1GCA: 5543.10
X_C1GCB: 352.57
X_C1GCC: 4104.32
X_C2GCA: 456.90
X_C2GCB: 6647.43
X_C2GCC: 7895.68
Total Profit: $3525000.04


## 6. Printing the outputs as strings, so they can be saved.

In [60]:
print(response.text)

## Mathematical Optimization Model for Production Planning

**Parameters:**

* **Availability of components:**
    *  A1 = 10,000 l (Available quantity of C1)
    *  A2 = 15,000 l (Available quantity of C2)
* **Minimum demand:**
    * D_GCA = 6,000 l (Minimum demand for GCA)
    * D_GCB = 7,000 l (Minimum demand for GCB)
    * D_GCC = 9,000 l (Minimum demand for GCC)
* **Critical element proportion:**
    * E1 = 0.4 (Proportion of critical element in C1)
    * E2 = 0.2 (Proportion of critical element in C2)
* **Critical element requirements:**
    * R_GCA = 0.3 (Minimum proportion of critical element in GCA)
    * R_GCB = 0.3 (Maximum proportion of critical element in GCB)
* **Component ratio:**
    * CR_GCC = 0.3 (Minimum ratio of C1 to C2 in GCC)
* **Profit per liter:**
    * P_GCA = $120 (Profit for GCA)
    * P_GCB = $135 (Profit for GCB)
    * P_GCC = $155 (Profit for GCC)

**Decision Variables:**

* **X_ij:** Liters of component i (i = 1 for C1, i = 2 for C2) used in product j (j

In [61]:
print(response2.text)

```python
import pyomo.environ as pyo

# Create a model
model = pyo.ConcreteModel()

# Define sets for components and products
components = ["C1", "C2"]
products = ["GCA", "GCB", "GCC"]

# Define parameters (using the provided sample data)
availability = {"C1": 10000, "C2": 15000}
demand = {"GCA": 6000, "GCB": 7000, "GCC": 9000}
critical_proportion = {"C1": 0.4, "C2": 0.2}
critical_requirement = {"GCA": 0.3, "GCB": 0.3}
component_ratio = {"GCC": 0.3}
profit = {"GCA": 120, "GCB": 135, "GCC": 155}

# Define decision variables
model.X = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Define the objective function
model.profit = pyo.Objective(
    expr=sum(
        profit[j] * (model.X[i, j] for i in components) for j in products
    ),
    sense=pyo.maximize,
)

# Define constraints
# 1. Availability constraints
model.availability_constraints = pyo.ConstraintList()
for i in components:
    model.availability_constraints.add(
        sum(model.X[i, j] for j in products) <= av